### Manipulation of PEAKS de novo results of Trocas 8 (low water, November 2019) non-incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 675) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
    cd /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/689_Trocas8_June2021_DENOVO_117/

/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/689_Trocas8_June2021_DENOVO_117


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks689 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/689_Trocas8_June2021_DENOVO_117/de novo peptides.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks689))

print(peaks689.columns)

# These columns mess things up- get rid of them

del peaks689['Fraction']
del peaks689['Scan']
del peaks689['Source File']
del peaks689['Tag Length']
del peaks689['PTM']
del peaks689['tag (>=0%)']
del peaks689['mode']
del peaks689['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks689.columns = columns

mean_len = peaks689['length'].mean()
print(mean_len)

# look at the dataframe
peaks689.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 82
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
8.926829268292684


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,LSSPATLNSR,98,10,523.2863,2,46.94,23600000.0,1044.5564,1.6
1,LSSPATLNSR,98,10,523.2861,2,46.44,23600000.0,1044.5564,1.1
2,LSSPATLNSR,98,10,523.2858,2,47.45,23600000.0,1044.5564,0.7
3,LSSPATLNSR,98,10,523.2859,2,47.95,23600000.0,1044.5564,0.8
4,SPATLNSR,98,8,423.2277,2,25.82,168000.0,844.4402,0.8


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [4]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks689['A'] = peaks689['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks689['C'] = peaks689['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks689['D'] = peaks689['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks689['E'] = peaks689['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks689['F'] = peaks689['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks689['G'] = peaks689['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks689['H'] = peaks689['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks689 output, there will be no isoleucines (they're lumped in with leucines)
peaks689['I'] = peaks689['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks689['K'] = peaks689['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks689['L'] = peaks689['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks689['M'] = peaks689['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks689['N'] = peaks689['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks689['P'] = peaks689['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks689['Q'] = peaks689['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks689['R'] = peaks689['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks689['S'] = peaks689['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks689['T'] = peaks689['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks689['V'] = peaks689['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks689['W'] = peaks689['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks689['Y'] = peaks689['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks689['c-carb'] = peaks689['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks689['m-oxid'] = peaks689['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks689['n-deam'] = peaks689['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks689['q-deam'] = peaks689['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks689['stripped_peptide'] = peaks689['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks689['stripped_length'] = peaks689['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks689['ptm-total'] = peaks689['c-carb'] + peaks689['m-oxid'] + peaks689['n-deam'] + peaks689['q-deam']

# calculate NAAF numerator for each peptide k
peaks689['NAAF_num.'] = peaks689['Area'] / peaks689['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks689.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/681_NMCP_50_GF75_DN/689_NMCP_50_GF75_DN50.csv")

# check out the results
peaks689.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,LSSPATLNSR,98,10,523.2863,2,46.94,23600000.0,1044.5564,1.6,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,2360000.0
1,LSSPATLNSR,98,10,523.2861,2,46.44,23600000.0,1044.5564,1.1,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,2360000.0
2,LSSPATLNSR,98,10,523.2858,2,47.45,23600000.0,1044.5564,0.7,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,2360000.0
3,LSSPATLNSR,98,10,523.2859,2,47.95,23600000.0,1044.5564,0.8,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,2360000.0
4,SPATLNSR,98,8,423.2277,2,25.82,168000.0,844.4402,0.8,1,...,0,0,0,0,0,0,SPATLNSR,8,0,21000.0


In [5]:
# keep only stripped peptide I/L and NAAF
dn_689 = peaks689[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_689.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_689.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/689_NMCP_50_GF75_DN50_ILnaafs.csv")

dn_689.head()

,stripped_peptide,Area,NAAF_num.
0,LSSPATLNSR,23600000.0,2360000.0
1,LSSPATLNSR,23600000.0,2360000.0
2,LSSPATLNSR,23600000.0,2360000.0
3,LSSPATLNSR,23600000.0,2360000.0
4,SPATLNSR,168000.0,21000.0


In [6]:
# made a new dataframe that contains the sums of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks689['A'].sum(),
        'C': peaks689['C'].sum(),
        'D': peaks689['D'].sum(),
        'E': peaks689['E'].sum(),
        'F': peaks689['F'].sum(),
        'G': peaks689['G'].sum(),
        'H': peaks689['H'].sum(),
        'I': peaks689['I'].sum(),
        'K': peaks689['K'].sum(),
        'L': peaks689['L'].sum(),
        'M': peaks689['M'].sum(),
        'N': peaks689['N'].sum(),
        'P': peaks689['P'].sum(),
        'Q': peaks689['Q'].sum(),
        'R': peaks689['R'].sum(),
        'S': peaks689['S'].sum(),
        'T': peaks689['T'].sum(),
        'V': peaks689['V'].sum(),
        'W': peaks689['W'].sum(),
        'Y': peaks689['Y'].sum(),
        'c-carb': peaks689['c-carb'].sum(),
        'm-oxid': peaks689['m-oxid'].sum(),
        'n-deam': peaks689['n-deam'].sum(),
        'q-deam': peaks689['q-deam'].sum(),
        'Total area': peaks689['Area'].sum(),
        'Total length': peaks689['stripped_length'].sum()
       }

totalpeaks689 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks689['% C w/ carb'] = totalpeaks689['c-carb'] / totalpeaks689['C'] 

# calculate percentage of M's that are oxidized
totalpeaks689['% M w/ oxid'] = totalpeaks689['m-oxid'] / totalpeaks689['M'] 

# calculate percentage of N's that are deamidated
totalpeaks689['% N w/ deam'] = totalpeaks689['n-deam'] / totalpeaks689['N'] 

# calculate percentage of N's that are deamidated
totalpeaks689['% Q w/ deam'] = totalpeaks689['q-deam'] / totalpeaks689['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks689['NAAF denom.'] = totalpeaks689['Total area'] / totalpeaks689['Total length']

# write modified dataframe to new txt file
totalpeaks689.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/681_NMCP_50_GF75_DN/689_NMCP_50_GF75_DN50_totals.csv")

totalpeaks689.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,73,15,20,23,16,27,8,0,48,99,...,7,3,3,318535200.0,715,1.0,0.212121,0.078947,0.107143,445503.776224


In [7]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 445503.776224

# use NAAF >50% ALC to get NAAF factor
peaks689['NAAF factor'] = (peaks689['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksAAPTM_689 = peaks689[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_689['A-NAAF50'] = peaksAAPTM_689['A'] * peaks689['NAAF factor']
peaksAAPTM_689['C-NAAF50'] = peaksAAPTM_689['C'] * peaks689['NAAF factor']
peaksAAPTM_689['D-NAAF50'] = peaksAAPTM_689['D'] * peaks689['NAAF factor']
peaksAAPTM_689['E-NAAF50'] = peaksAAPTM_689['E'] * peaks689['NAAF factor']
peaksAAPTM_689['F-NAAF50'] = peaksAAPTM_689['F'] * peaks689['NAAF factor']
peaksAAPTM_689['G-NAAF50'] = peaksAAPTM_689['G'] * peaks689['NAAF factor']
peaksAAPTM_689['H-NAAF50'] = peaksAAPTM_689['H'] * peaks689['NAAF factor']
peaksAAPTM_689['I-NAAF50'] = peaksAAPTM_689['I'] * peaks689['NAAF factor']
peaksAAPTM_689['K-NAAF50'] = peaksAAPTM_689['K'] * peaks689['NAAF factor']
peaksAAPTM_689['L-NAAF50'] = peaksAAPTM_689['L'] * peaks689['NAAF factor']
peaksAAPTM_689['M-NAAF50'] = peaksAAPTM_689['M'] * peaks689['NAAF factor']
peaksAAPTM_689['N-NAAF50'] = peaksAAPTM_689['N'] * peaks689['NAAF factor']
peaksAAPTM_689['P-NAAF50'] = peaksAAPTM_689['P'] * peaks689['NAAF factor']
peaksAAPTM_689['Q-NAAF50'] = peaksAAPTM_689['Q'] * peaks689['NAAF factor']
peaksAAPTM_689['R-NAAF50'] = peaksAAPTM_689['R'] * peaks689['NAAF factor']
peaksAAPTM_689['S-NAAF50'] = peaksAAPTM_689['S'] * peaks689['NAAF factor']
peaksAAPTM_689['T-NAAF50'] = peaksAAPTM_689['T'] * peaks689['NAAF factor']
peaksAAPTM_689['V-NAAF50'] = peaksAAPTM_689['V'] * peaks689['NAAF factor']
peaksAAPTM_689['W-NAAF50'] = peaksAAPTM_689['W'] * peaks689['NAAF factor']
peaksAAPTM_689['Y-NAAF50'] = peaksAAPTM_689['Y'] * peaks689['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaksAAPTM_689['ccarb-NAAF50'] = peaksAAPTM_689['c-carb'] * peaksAAPTM_689['NAAF factor']
peaksAAPTM_689['moxid-NAAF50'] = peaksAAPTM_689['m-oxid'] * peaksAAPTM_689['NAAF factor']
peaksAAPTM_689['ndeam-NAAF50'] = peaksAAPTM_689['n-deam'] * peaksAAPTM_689['NAAF factor']
peaksAAPTM_689['qdeam-NAAF50'] = peaksAAPTM_689['q-deam'] * peaksAAPTM_689['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_689.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/681_NMCP_50_GF75_DN/689_NMCP_50_GF75_DN50_naaf.csv")

peaksAAPTM_689.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,LSSPATLNSR,5.297374,1,0,0,0,0,0,0,0,...,5.297374,15.892121,5.297374,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LSSPATLNSR,5.297374,1,0,0,0,0,0,0,0,...,5.297374,15.892121,5.297374,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LSSPATLNSR,5.297374,1,0,0,0,0,0,0,0,...,5.297374,15.892121,5.297374,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,LSSPATLNSR,5.297374,1,0,0,0,0,0,0,0,...,5.297374,15.892121,5.297374,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,SPATLNSR,0.047138,1,0,0,0,0,0,0,0,...,0.047138,0.094275,0.047138,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMs

index = ['sample total']

data = {'NAAF': peaksAAPTM_689['NAAF factor'].sum(),
        'A': peaksAAPTM_689['A-NAAF50'].sum(),
        'C': peaksAAPTM_689['C-NAAF50'].sum(),
        'D': peaksAAPTM_689['D-NAAF50'].sum(),
        'E': peaksAAPTM_689['E-NAAF50'].sum(),
        'F': peaksAAPTM_689['F-NAAF50'].sum(),
        'G': peaksAAPTM_689['G-NAAF50'].sum(),
        'H': peaksAAPTM_689['H-NAAF50'].sum(),
        'I': peaksAAPTM_689['I-NAAF50'].sum(),
        'K': peaksAAPTM_689['K-NAAF50'].sum(),
        'L': peaksAAPTM_689['L-NAAF50'].sum(),
        'M': peaksAAPTM_689['M-NAAF50'].sum(),
        'N': peaksAAPTM_689['N-NAAF50'].sum(),
        'P': peaksAAPTM_689['P-NAAF50'].sum(),
        'Q': peaksAAPTM_689['Q-NAAF50'].sum(),
        'R': peaksAAPTM_689['R-NAAF50'].sum(),
        'S': peaksAAPTM_689['S-NAAF50'].sum(),
        'T': peaksAAPTM_689['T-NAAF50'].sum(),
        'V': peaksAAPTM_689['V-NAAF50'].sum(),
        'W': peaksAAPTM_689['W-NAAF50'].sum(),
        'Y': peaksAAPTM_689['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_689['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_689['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_689['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_689['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 445503.776224

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/681_NMCP_50_GF75_DN/689_NMCP_50_GF75_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,83.393791,68.081438,1.046324,1.495439,0.309831,1.3823,4.375138,0.069225,0.0,2.849945,...,0.426384,1.046324,0.950845,0.115883,0.243096,1.0,0.28417,0.004204,0.34861,0.000187


## Export stripped peptides >50% ALC

In [9]:
###### keep only stripped peptide column 
pep50 = peaks689[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/681_NMCP_50_GF75_DN/689_NMCP_50_GF75_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/681_NMCP_50_GF75_DN/689_NMCP_50_GF75_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/681_NMCP_50_GF75_DN/689_NMCP_50_GF75_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks689['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks689[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 82
# nonredundant stripped Peaks peptides >50% ALC 60
average peptide length Peaks peptides >50% ALC 8.71951219512195
# redundant Peaks peptides >50% ALC 82
# nonredundant Peaks peptides 60


,stripped_peptide
0,LSSPATLNSR
1,LSSPATLNSR
2,LSSPATLNSR
3,LSSPATLNSR
4,SPATLNSR
